# Object Classification Using CNN -Acc 0.8235

###### content

    1. Introduction
    2. Data preparation
        2.1 Load data
        2.2 Reshape
        2.3 Label encoding
        2.4 Split training and valdiation set
    3. CNN
        3.1 Define the model
        3.2 Set the optimizer
    4. Evaluate the model
        4.1 Training and validation 
    5. Prediction 
        5.1 Predict results
    6. Confusion matrix

###### 1.Introduction

This is a 5 layers Sequential Convolutional Neural Network for object classification trained on the dataset downloaded from IKEA site (https://www.ikea.com/). I choosed to build it with keras API (Tensorflow backend) which is very intuitive. Firstly, I will prepare the data then i will focus on the CNN modeling and evaluation.

I achieved 82.35% of accuracy with this CNN trained.


This Notebook follows three main parts:

The data preparation
The CNN modeling and evaluation
The results prediction
The Confusion matrix generation

IMPORT LIBRARIES

In [1]:
from __future__ import print_function
import keras
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Input,UpSampling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Model
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.layers import merge
from keras.layers import Reshape
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

DATADIR="/content/drive/My Drive/dataset"
CATEGORIES=["2.dining_chairs","3.beds","4.sofa","5.storage_furniture"]

Using TensorFlow backend.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###### 2.Data Preparation

    The dataset will be read in above cell from the path DATADIR. There are four classes in the dataset.             Those are:  
        1.Dining chairs
        2.Beds
        3.Sofa
        4.Storage_furniture
    

In [2]:
training_data=[]
def create_training_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR,category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array=cv2.resize(img_array,(400,400))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
create_training_data()
print(len(training_data))

421


Shuffled the data randomly and I saved the features and lables to the arrays x and y respectively.

In [0]:
import random
random.shuffle(training_data)
x=[]
y=[]
for features,label in training_data:
    x.append(features)
    y.append(label)
x=np.array(x).reshape(-1,400,400,1)


Splitting the dataset into training, testing and validation sets.
    
    train set :- 60 % of dataset
    test set :- 40 % of dataset
    validation set :- 50 % of test dataset


In [4]:
train_images, test_images, train_labels, test_labels = train_test_split(x, y, test_size=0.4, random_state=42)
print('Training data shape : ', train_images.shape, len(train_labels))
print('Testing data shape : ', test_images.shape, len(test_labels))
classes = np.unique(train_labels)
#classes=np.append(classes,0)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)
train_data = train_images.astype('float32')
test_data = test_images.astype('float32')
train_data /= 255
test_data /= 255
print(len(train_labels))
print(len(test_labels))
train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)
print('Original label : ', train_labels[0])
print('After conversion to categorical ( one-hot ) : ', train_labels_one_hot[0])
validation_images, test_images, validation_labels, test_labels = train_test_split(test_data,test_labels_one_hot, test_size=0.5, random_state=42)

Training data shape :  (252, 400, 400, 1) 252
Testing data shape :  (169, 400, 400, 1) 169
Total number of outputs :  4
Output classes :  [0 1 2 3]
252
169
Original label :  3
After conversion to categorical ( one-hot ) :  [0. 0. 0. 1.]


###### 3.CNN

3.1 Define the model

I used the Keras Sequential API, where you have just to add one layer at a time, starting from the input.

Here I used 5 convolution layers. The first is the convolutional (Conv2D) layer. It is like a set of learnable filters. I choosed to set 32 filters for the first three conv2D layers and 64 filters for the next ones. The last conv2D layer have 128 filters. Each filter transforms a part of the image (defined by the kernel size) using the kernel filter. The kernel filter matrix is applied on the whole image. Filters can be seen as a transformation of the image.

The second important layer in CNN is the pooling (MaxPool2D) layer. This layer simply acts as a downsampling filter. It looks at the 2 neighboring pixels and picks the maximal value. These are used to reduce computational cost, and to some extent also reduce overfitting. We have to choose the pooling size (i.e the area size pooled each time) more the
pooling dimension is high, more the downsampling is important.

Combining convolutional and pooling layers, CNN are able to combine local features and learn more global features of the image.

Dropout is a regularization method, where a proportion of nodes in the layer are randomly ignored (setting their wieghts to zero) for each training sample. This technique also improves generalization and reduces the overfitting.
'Relu' is the rectifier (activation function max(0,x). The rectifier activation function is used to add non linearity to the network.
The Flatten layer is use to convert the final feature maps into a one single 1D vector. This flattening step is needed so that you can make use of fully connected layers after some convolutional/maxpool layers. It combines all the found local features of the previous convolutional layers.
In the end I used the features in two fully-connected (Dense) layers which is just artificial an neural networks (ANN) classifier. In the last layer(Dense(10,activation="softmax")) the net outputs distribution of probability of each class.
    
Once our layers are added to the model, we need to set up a score function, a loss function and an optimisation algorithm.
I used a specific form for categorical classifications (>2 classes) called the "categorical_crossentropy" as a loss function.

I choosed Adam as optimizer, it is a Straightforward to implement,Computationally efficient, Little memory requirements and Well suited for problems that are large in terms of data and/or parameters.


In [5]:
cnn3 = Sequential()
cnn3.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(400, 400, 1)))
cnn3.add(MaxPooling2D((2, 2)))
cnn3.add(Dropout(0.25))

cnn3.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(400, 400, 1)))
cnn3.add(MaxPooling2D((2, 2)))
cnn3.add(Dropout(0.25))
cnn3.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(400, 400, 1)))
cnn3.add(MaxPooling2D((2, 2)))
cnn3.add(Dropout(0.25))

cnn3.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn3.add(MaxPooling2D(pool_size=(2, 2)))
cnn3.add(Dropout(0.25))

cnn3.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn3.add(Dropout(0.4))

cnn3.add(Flatten())

cnn3.add(Dense(128, activation='relu'))
cnn3.add(Dropout(0.5))
cnn3.add(Dense(4, activation='softmax'))

cnn3.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
cnn3.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 398, 398, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 199, 199, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 199, 199, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 197, 197, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 98, 98, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)  

When I train the model for a given number of epochs (iterations on a dataset) with batch size 1.

In [6]:
cnn3.fit(train_data,train_labels_one_hot,epochs=20,batch_size=1,verbose=1,validation_data=(validation_images, validation_labels))

Instructions for updating:
Use tf.cast instead.
Train on 252 samples, validate on 84 samples
Epoch 1/20
252/252 [==============================] - 7s 29ms/step - loss: 1.4083 - acc: 0.3690 - val_loss: 1.3298 - val_acc: 0.4405
Epoch 2/20
252/252 [==============================] - 3s 14ms/step - loss: 1.3588 - acc: 0.4206 - val_loss: 1.3603 - val_acc: 0.4405
Epoch 3/20
252/252 [==============================] - 4s 14ms/step - loss: 1.2752 - acc: 0.4881 - val_loss: 1.2125 - val_acc: 0.5119
Epoch 4/20
252/252 [==============================] - 4s 14ms/step - loss: 1.0283 - acc: 0.5873 - val_loss: 1.1354 - val_acc: 0.5714
Epoch 5/20
252/252 [==============================] - 4s 14ms/step - loss: 0.9040 - acc: 0.6429 - val_loss: 1.1060 - val_acc: 0.5357
Epoch 6/20
252/252 [==============================] - 4s 14ms/step - loss: 0.7562 - acc: 0.6944 - val_loss: 0.9518 - val_acc: 0.6190
Epoch 7/20
252/252 [==============================] - 4s 14ms/step - loss: 0.6817 - acc: 0.7500 - val_loss: 1

###### 4.Evaluate the model

When I evaluate on testing data:
    
    Got 82.35 % of accuracy

In [7]:
cnn3.evaluate(test_images, test_labels,verbose=0)

[1.443966994566076, 0.8235294131671681]

Model saved in my drive

In [0]:
cnn3.save_weights('/content/drive/My Drive/dataset/model2.h5')

###### 5.Prediction

Now, with the saved model I predict the classification of test images by using predict function

In [0]:
predicted=cnn3.predict(test_images)

###### 6.Confusion Matrix Generation

Function for obtaining lables in an array

In [0]:
import numpy 
x=0
y=[]
# Function to get max element 
def maxelement(arr): 
	
	# get number of rows and columns 
	no_of_rows = len(arr) 
	no_of_column = len(arr[0]) 
	
	for i in range(no_of_rows): 
		
		# Initialize max1 to 0 at beginning 
		# of finding max element of each row 
		max1 = 0
		for j in range(no_of_column): 
			if arr[i][j] > max1 : 
				max1 = arr[i][j] 
				x=j
		# print maximum element of each row  
		y.append(x)
# Driver Code 
	return(y)

I applied the above function for predicted and actual lables

In [11]:
a=maxelement(predicted)
b=maxelement(test_labels)
print(a)
print(b)

[0, 2, 3, 2, 1, 0, 0, 2, 1, 1, 2, 0, 0, 3, 2, 3, 0, 0, 3, 3, 2, 0, 2, 3, 3, 1, 3, 1, 3, 0, 3, 3, 2, 2, 2, 3, 2, 3, 1, 3, 0, 0, 3, 2, 3, 2, 1, 3, 2, 2, 1, 0, 2, 2, 2, 0, 2, 1, 0, 3, 1, 2, 3, 2, 2, 2, 2, 3, 1, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 2, 0, 3, 2, 0, 2, 0, 2, 3, 2, 1, 0, 0, 3, 3, 1, 2, 0, 0, 3, 3, 3, 0, 0, 3, 3, 2, 3, 2, 3, 3, 1, 3, 3, 2, 1, 3, 3, 2, 1, 2, 3, 2, 3, 1, 3, 0, 0, 3, 2, 3, 2, 1, 3, 3, 2, 1, 0, 2, 3, 2, 1, 2, 1, 2, 3, 1, 2, 2, 2, 3, 2, 2, 3, 1, 2, 0, 2, 0, 0, 1, 1, 3, 0, 3, 2, 0, 3, 2, 0, 2]
[0, 2, 3, 2, 1, 0, 0, 2, 1, 1, 2, 0, 0, 3, 2, 3, 0, 0, 3, 3, 2, 0, 2, 3, 3, 1, 3, 1, 3, 0, 3, 3, 2, 2, 2, 3, 2, 3, 1, 3, 0, 0, 3, 2, 3, 2, 1, 3, 2, 2, 1, 0, 2, 2, 2, 0, 2, 1, 0, 3, 1, 2, 3, 2, 2, 2, 2, 3, 1, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 2, 0, 3, 2, 0, 2, 0, 2, 3, 2, 1, 0, 0, 3, 3, 1, 2, 0, 0, 3, 3, 3, 0, 0, 3, 3, 2, 3, 2, 3, 3, 1, 3, 3, 2, 1, 3, 3, 2, 1, 2, 3, 2, 3, 1, 3, 0, 0, 3, 2, 3, 2, 1, 3, 3, 2, 1, 0, 2, 3, 2, 1, 2, 1, 2, 3, 1, 2, 2, 2, 3, 2, 2, 3, 1, 2, 0, 2, 0, 0, 1, 1, 3, 0,

I gave the above outputs to the input of confusion matrix function, The result is the confusion matrix.

In [12]:
# Python script for confusion matrix creation. 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
results = confusion_matrix(b, a) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(b, a))
print('Report : ')
print(classification_report(b, a)) 



Confusion Matrix :
[[36  0  0  0]
 [ 0 26  0  0]
 [ 0  0 56  0]
 [ 0  0  0 52]]
Accuracy Score : 1.0
Report : 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00        26
           2       1.00      1.00      1.00        56
           3       1.00      1.00      1.00        52

   micro avg       1.00      1.00      1.00       170
   macro avg       1.00      1.00      1.00       170
weighted avg       1.00      1.00      1.00       170

